### 新古典派成長モデルの時間反復法による数値解法
代表的個人の問題は以下のように書ける
\begin{align*}
 && \underset{{c_{t},k_{t+1}}}{\max} \sum_{t=0}^{\infty} \beta^t u(c_t) \\
 \text{subject to} && c_t + k_{t+1} = f(k_t), \\
 && k_0 \text{ is given.}
\end{align*}
ここで、$u(c)=\frac{c^{1-\sigma}}{1-\sigma}, f(k)=k^{\alpha}+(1-\delta)k$

In [1]:
using LinearAlgebra
using Statistics # for mean
# using Roots
using NLsolve
using Printf

In [2]:
struct Model{TI<:Integer, TF<:AbstractFloat, TV<:Vector}
    β::TF       # 割引因子
    γ::TF       # 相対的危険回避度
    α::TF       # 資本分配率
    δ::TF       # 固定資本減耗      
    # 定常状態の値
    ykss::TF
    kss::TF
    yss::TF
    css::TF
    nk::TI      # 資本グリッドの個数
    kmax::TF    # 資本グリッドの最大値
    kmin::TF    # 資本グリッドの最小値 (0にすると生産が出来なくなる)
    kgrid::TV   # 資本グリッド
    T::Array{Float64,2}
    invT::Array{Float64,2}
    maxiter::TI # 繰り返し計算の最大値
    tol::TF     # 許容誤差
end

### チェビシェフ多項式
基底関数$T_i(x):[-1,1]\rightarrow[-1,1]$を組み合わせた$N-1$次の多項式を用いて、$N$個の点の間を補間する
\begin{equation*}
g(x;\theta) = \theta_0 + \theta_1 T_1(x) + \theta_2 T_2(x) + \cdots + \theta_{N-1} T_{N-1}(x)
\end{equation*}
ここで、
\begin{align*}
T_0(x) = 1, \\
T_1(x) = x, \\
T_2(x) = 2x^2-1, \\
\vdots \\
T_{N-1}(x) = 2xT_{N-2}(x) - T_{N-3}(x)
\end{align*}


ここでは、政策関数$c=h(k)$をチェビシェフ多項式で近似する

\begin{equation*}
g(k;\theta) = \theta_0 + \theta_1 T_1(\varphi(k)) + \theta_2 T_2(\varphi(k)) + \cdots + \theta_{N-1} T_{N-1}(\varphi(k))
\end{equation*}


In [3]:
function polybas(kmin,kmax,Np,kgrid)
    
    # Np: 多項式の次数-1
    # Ng: グリッドの数
    Ng = size(kgrid,1)
    x = (2/(kmax-kmin))*(kgrid .- kmin) .- 1
    
    # 基底関数の行列(NgxNp)を再帰的に求める
    T = zeros(Ng,Np)
    T0 = ones(Ng)
    T1 = x
    T2 = 2*x.*T1 - T0
    T[:,1] = T1
    T[:,2] = T2
    
    for i=3:Np-1
        T[:,i] = 2*x.*T[:,i-1] - T[:,i-2] 
    end
    
    T = [T0 T[:,1:(Np-1)]]
    
    return T
    
end

polybas (generic function with 1 method)

### 評価点
#### チェビシェフゼロ点
\begin{align*}
x_0 = 0, \\
x_j = \cos \left( \frac{(2j-1)\pi}{2(N-1)} \right) \text{ for } j=1,\dots,N-1
\end{align*}
#### チェビシェフ極値点
\begin{equation*}
x_j = \cos \left( \frac{j\pi}{N-1} \right) \text{ for } j=0,1,\dots,N-1
\end{equation*}

ここで、より一般的な関数では、$k$の値は$[k_1,k_N]$の間にあるとすると、$k$から$x\in[-1,1]$への変換は

\begin{equation*}
x=\varphi(k)=\frac{2(k-k_1)}{k_N-k_1}-1
\end{equation*}

この変換の逆は以下で与えられる

\begin{equation*}
k=\varphi^{-1}(x)=k_1+0.5(1+x)(k_N-k_1)
\end{equation*}

In [4]:
function polygrid(kmin,kmax,N)
    
    temp = collect(LinRange(0,N-1,N))
    x = -cos.((pi/(N-1))*temp) # チェビシェフ極値点
    #temp = collect(LinRange(1,N-1,N-1))
    #x = [0; -cos((pi/2/(N-1))*(2*temp .- 1))] # チェビシェフゼロ点
    
    # xからkに変換
    k = 0.5*(kmax-kmin)*(x .+ 1) .+ kmin
   
    return k
    
end

polygrid (generic function with 1 method)

### オイラー方程式
オイラー方程式は以下で与えられる
\begin{equation*}
u^{\prime}(c_{t}) = \beta u^{\prime}(c_{t+1})f^{\prime}(k_{t+1})
\end{equation*}

ここで、$u^{\prime}(c)=c^{-\sigma}, f^{\prime}(k)=\alpha k^{\alpha-1}+(1-\delta)$

In [5]:
function EulerEq_cheb(x,m,capital,theta)
# cを与えたときのオイラー方程式の残差を返す関数
    
    cons = x[1]
    wealth = capital^m.α + (1-m.δ)*capital
    
    kprime = wealth - cons
    # トリック: k'は正の値しか取らない
    kprime = max(m.kgrid[1],kprime)

    T = polybas(m.kmin,m.kmax,m.nk,[kprime]) # the type of [kprime] is Array{Float64,1}
    temp = T*theta
    cnext = temp[1]
    # オイラー方程式
    res = mu_CRRA(cons,m.γ) - m.β*mu_CRRA(cnext,m.γ)*(m.α*kprime.^(m.α-1) + (1-m.δ))

    return res 
end

EulerEq_cheb (generic function with 1 method)

In [6]:
function CRRA(cons::Real, gamma)
    """
    Compute CRRA utility function
    
    # Arguments

    - `cons::Real`: consumption value
    - `gamma::Real`: relative risk aversion
    
    # Return 
    - `util::Real`: utility value 
    """
    if gamma != 1.0
        util = cons^(1.0 - gamma) / (1.0 - gamma)
    else
        util = log(cons) 
    end
    return util
end

CRRA (generic function with 1 method)

In [7]:
function mu_CRRA(cons::Real, gamma)
    """
    Compute marginal utility of CRRA-type function
    
    # Arguments 
    - "cons::VecOrMat": consumption value
    - "gamma::Real": relative risk aversion
    
    # Return
    - "mu::Real": marginal utility 
    """
    
    mu = cons^-gamma
    return mu
end

mu_CRRA (generic function with 1 method)

### アルゴリズム
#### 1. グリッド生成
a. 状態空間を有限個のグリッド$\{k_1,k_2,\dots,k_N\}$に区切る

b. グリッド上の政策関数の初期値$c=h(k_i)$を当て推量する

#### 2. 収束の基準
収束の基準になるパラメータ$\varepsilon$を与える

#### 3. 近似・評価
グリッドの点$k_i$上にない政策関数の値については、チェビシェフ多項式を使って補間する

\begin{equation*}
g(k;\theta) = \theta_0 + \theta_1 T_1(\varphi(k)) + \theta_2 T_2(\varphi(k)) + \cdots + \theta_{N-1} T_{N-1}(\varphi(k))
\end{equation*}

をパラメータのベクトル$\mathbf{\theta}$を使って近似したときの、$k$上での政策関数の値とする

#### 4. 最適化
古い政策関数$h^{(n-1)}(k)$を所与として、各$k_i$について、
\begin{equation*}
u^{\prime}(c) = \beta u^{\prime} \left( h^{(n-1)}(f(k_i)-c;\mathbf{\theta}) \right) f^{\prime}(f(k_i)-c)
\end{equation*}
を$c$について解く

このステップで、新しい政策関数$c=h^{(n)}(k)$を得る

#### 5. 収束の確認
全ての$k_i$について、$\| h^{(n)}(k_i)-h^{(n-1)}(k_i) \| < \varepsilon$であればストップ

そうでなければ、$h^{(n)}(k)$を$h^{(n-1)}(k)$に代入して、ステップ3とステップ4を繰り返す

In [8]:
function nti_cheb(m::Model)
    
    # *** 収束の基準 ***
    it = 1         # ループ・カウンター
    dif2 = 1.0     # 政策関数の繰り返し誤差
    tolfun = 1e-10 # NLsolveのオプション(最適化の許容誤差)

#     println(" ")
#     println("-+- Solve a neoclassical growth model with time iteration -+-")
#     println(" ")    
    
    ## STEP 1(b): 政策関数の初期値を当て推量    
    cfcn0 = m.kgrid
    #cfcn0 = m.css/m.kss*m.kgrid
    cfcn1 = zeros(m.nk)
    
    ## STEP 4: 価値関数を繰り返し計算
    while (it < m.maxiter && dif2 > m.tol)
        
        # 次期の政策関数を補間
        # 多項式のフィット
        theta = m.invT*cfcn0
        
        for i = 1:m.nk
            
            capital = m.kgrid[i]
            wealth = capital.^m.α + (1-m.δ).*capital
            
            # Juliaの最適化関数(NLsolve or Roots)を使って各グリッド上の政策関数の値を探す
            EulerEq_cheb!(x) = EulerEq_cheb(x,m,capital,theta) # x is a vector (for NLsolve)
#             EulerEq_cheb!(cons) = EulerEq_cheb([cons],m,capital,theta) # cons is a scalar (for Roots)
            # 最適化の初期値は古い政策関数の値
            # for Roots
#             cons = find_zero(EulerEq_cheb!,cfcn0[i])            
#             cfcn1[i] = cons
            # for NLsolve
            sol = nlsolve(EulerEq_cheb!,[cfcn0[i]],ftol=tolfun)
            cfcn1[i] = sol.zero[1]
            # グリッドごとに最適化の結果を確認
            #disp([cons capital wealth kprime]);
            #pause

        end
        
        # 繰り返し計算誤差を確認
        dif2 = maximum(abs.(cfcn1-cfcn0)) # 政策関数の繰り返し計算誤差(図示のため)
        
#         println([it dif2])
        
        # 政策関数をアップデート
        cfcn0 = copy(cfcn1)
        
        it = it + 1
        
    end
        
    return cfcn0
end

nti_cheb (generic function with 1 method)

数値解により得られた政策関数$c=h(k;\mathbf{\theta})$をオイラー方程式に代入して、誤差
\begin{equation*}
\frac{\beta u^{\prime} \left( h(\underbrace{f(k)-h(k;\mathbf{\theta})}_{k^{\prime}};\mathbf{\theta}) \right) f^{\prime}(\underbrace{f(k)-h(k;\mathbf{\theta})}_{k^{\prime}})
}{u^{\prime}(h(k;\mathbf{\theta}))}-1
\end{equation*}
を計算できる。近似に用いたグリッド上では計算誤差はほぼゼロに等しい

In [9]:
function calcerr(m,cfcn0)
## オイラー方程式から誤差を測定
    # 元のグリッドではオイラー方程式の誤差はゼロになるため、グリッドを細かくとる
    kgrid_err = collect(LinRange(m.kmin,m.kmax,(m.nk-1)*10+1))
    # 多項式のフィット
    theta = m.invT*cfcn0
    # 各kにおける基底関数の計算
    T = polybas(m.kmin,m.kmax,m.nk,kgrid_err)
    # c=h(k;b)の値を近似
    cons = T*theta
    LHS  = mu_CRRA.(cons,m.γ)

    # k'=f(k)-c
    kp   = kgrid_err.^m.α + (1-m.δ)*kgrid_err - cons
    # 各k'における基底関数の計算
    T = polybas(m.kmin,m.kmax,m.nk,kp)
    # c'=h(k';b)の値を近似
    cnext = T*theta
    # f'(k')
    rent = m.α.*kp.^(m.α-1.0) .- m.δ
    RHS  = m.β.*(1 .+ rent).*mu_CRRA.(cnext,m.γ)

    err  = RHS./LHS.-1.0
    
    return err
    
end

calcerr (generic function with 1 method)

### メインファイル

In [10]:
# カリブレーション
β = 0.96 # 割引因子
γ = 1.0  # 相対的危険回避度(異時点間の代替の弾力性の逆数)
α = 0.40 # 資本分配率
δ = 1.0  # 固定資本減耗(delta=1.0のときは解析解が存在)

# 定常状態の値
ykss = (1/β-1+δ)/α
kss = ykss^(1/(α-1))
yss = ykss*kss
css = yss-δ*kss

# kmax = 0.5  # 資本グリッドの最大値
# kmin = 0.05 # 資本グリッドの最小値 (0にすると生産が出来なくなる)
kmax = 1.2*kss # 資本グリッドの最大値
kmin = 0.8*kss # 資本グリッドの最小値 (0にすると生産が出来なくなる)

maxiter = 1000 # 繰り返し回数の最大値
tol = 1.0e-8;  # 許容誤差(STEP 2)

多項式のフィット

\begin{equation*}
\theta=T(x)^{-1}g(x)
\end{equation*}

において、行列$T(x)$は一度評価点と基底関数を定めるとその後は固定される。逆行列の計算には時間がかかるため、多項式近似を時間反復法に適用するときは、逆行列の計算をアルゴリズムの初期にあらかじめ行うとよい

In [13]:
norms = zeros(3,2)
times = zeros(3,2)

nkvec = [3 5 9]';

for i=1:3

    ## STEP 1(a): グリッド生成
    nk = nkvec[i] # グリッドの数
    kgrid = polygrid(kmin,kmax,nk) # チェビシェフ評価点
    T = polybas(kmin,kmax,nk,kgrid) # 基底関数
    invT = inv(T) # 逆行列をあらかじめ計算しておく
    
    # 構造体にパラメータを格納
    m = Model(β,γ,α,δ,ykss,kss,yss,css,nk,kmax,kmin,kgrid,T,invT,maxiter,tol)

    # time iteration
    times[i,1] = @elapsed cfcn0 = nti_cheb(m)
    
    # 計算誤差
    err = calcerr(m,cfcn0)
    # 平均値、最大値のlog10
    norms[i,:] = log10.([mean(abs.(err)) maximum(abs.(err))])
    
end

# 相対的な計算時間
times[:,2] = times[:,1]./times[1,1];

In [14]:
# print tables
println(norms)
println(times)

[-3.49855 -3.23498; -5.79966 -5.48986; -7.67968 -7.6768]
[0.00245278 1.0; 0.00604298 2.46372; 0.0143035 5.83155]
